In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
import os
import warnings

plt.rc('font', family='D2Coding')
plt.rcParams['axes.unicode_minus'] = False

BASE_PATH = r"C:\Users\sangh\OneDrive\바탕 화면\AI\Ecommerce-Data-Analysis-Practice\data"

FILE_LIST = {
    'Sale_Change': ("'25.12월 4주 주간아파트가격동향조사 시계열_매매변동률.csv", False),
    'Jeonse_Change': ("'25.12월 4주 주간아파트가격동향조사 시계열_전세변동률.csv", False),
    'Sale_Index': ("'25.12월 4주 주간아파트가격동향조사 시계열_매매지수.csv", False),
    'Jeonse_Index': ("'25.12월 4주 주간아파트가격동향조사 시계열_전세지수.csv", False),
    'Scale_Sale': ("'25.12월 4주 주간아파트가격동향조사 시계열_규모별매매변동률.csv", True),
    'Scale_Jeonse': ("'25.12월 4주 주간아파트가격동향조사 시계열_규모별전세변동률.csv", True),
    'Age_Sale': ("'25.12월 4주 주간아파트가격동향조사 시계열_연령별매매변동률.csv", True),
    'Age_Jeonse': ("'25.12월 4주 주간아파트가격동향조사 시계열_연령별전세변동률.csv", True),
}

def parse_custom_date(date_series):
    dates = []
    current_year = 2012
    last_month = 5

    for item in date_series:
        if pd.isna(item):
            dates.append(pd.NaT)
            continue
        item = str(item).strip().replace("'", "")

        if item.count('.') == 2:
            parts = item.split('.')
            y, m, d = int(parts[0]), int(parts[1]), int(parts[2])
            current_year = 2000 + y
            last_month = m
            dates.append(pd.Timestamp(year=current_year, month=m, day=d))
            continue

        parts = item.split('.')
        m, d = int(parts[0]), int(parts[1])
        if m < last_month:
            current_year += 1
        last_month = m
        dates.append(pd.Timestamp(year=current_year, month=m, day=d))
    return dates

def load_and_melt(filename, value_name, is_category=False):
    file_path = os.path.join(BASE_PATH, filename)
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없음: {filename}")
        return pd.DataFrame()
    try:
        nrows = 4 if is_category else 3
        try:
            df_header = pd.read_csv(file_path, encoding='cp949', header=None, nrows=nrows)
        except:
            df_header = pd.read_csv(file_path, encoding='utf-8', header=None, nrows=nrows)
        
        if is_category:
            row_region = df_header.iloc[1]
            row_sub = df_header.iloc[3]
        else:
            row_region = df_header.iloc[1]
            row_sub = df_header.iloc[2]
        
        cols = []
        current_region = None
        for i in range(len(row_region)):
            r1 = row_region[i]
            sub = row_sub[i]

            if pd.notna(r1):
                clean_region = str(r1).replace('\n', '').split('(')[0].strip()
                current_region = clean_region

            if pd.notna(sub):
                clean_sub = str(sub).replace('\n', '').strip()
                if current_region:
                    if clean_sub == current_region:
                        col_name = current_region
                    else:
                        col_name = f"{current_region}_{clean_sub}"
                else:
                    col_name = clean_sub
            else:
                col_name = current_region if current_region else f"Unknown_{i}"
            cols.append(col_name)

        try:
            df = pd.read_csv(file_path, encoding='cp949', header=None, skiprows=5)
        except:
            df = pd.read_csv(file_path, encoding='utf-8', header=None, skiprows=5)

        actual_cols = len(df.columns)
        parsed_cols = len(cols)
        if actual_cols < parsed_cols: cols = cols[:actual_cols]
        elif actual_cols > parsed_cols: df = df.iloc[:, :parsed_cols]

        df['Date'] = parse_custom_date(df[0])

        df.columns = ['Date_Raw'] + cols[1:]
        df = df.drop(columns=['Date_Raw'])

        df_melted = df.melt(id_vars=['Date'], var_name='Region', value_name=value_name)
        return df_melted.dropna(subset=['Date'])
    
    except Exception as e:
        print(f"⚠️ {filename} 처리 중 오류: {e}")
        return pd.DataFrame()    
